# Prelude

In [23]:
import numpy
import scipy
import matplotlib
import seaborn

import torch
import torch.nn as nn
import datasets
import wandb

import random


import nltk.lm as lm
from nltk import ngrams
from nltk import tokenize
from nltk.corpus import stopwords, gutenberg
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize

from matplotlib import pyplot

In [2]:
import pickle

# Let's print something the shape or whatever

In [3]:
dataset = datasets.load_dataset(
    'cnn_dailymail',
    '3.0.0',
)

In [4]:
train = dataset['train']
test = dataset['test']
val = dataset['validation']

In [5]:
print(train.shape)
print(test.shape)
print(val.shape)

(287113, 3)
(11490, 3)
(13368, 3)


In [6]:
print(str(train.data[0][0])[:100])
print(str(train.data[1][0])[:100])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 milli
Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .
Young actor says he has


# Fun with NLTK

## Tokenising text

In [7]:
train_text_raw = [tokenize.word_tokenize(str(x)) for x in train.data[0]]
train_high_raw = [tokenize.word_tokenize(str(x)) for x in train.data[1]]

In [ ]:
pickle.dump(train_text_raw, open('train_text_raw.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)
pickle.dump(train_high_raw, open('train_high_raw.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)

## Cleaning text

In [6]:
train_text_raw = pickle.load(open('train_text_raw.pickle', 'rb'))
train_high_raw = pickle.load(open('train_high_raw.pickle', 'rb'))

In [8]:
words_raw = set.union(*[set(x) for x in train_text_raw]) | set.union(*[set(x) for x in train_high_raw])

In [9]:
stop_words = stopwords.words('english')
# punctuation = set(x for x in sum(train_text, []) + sum(train_high, []) if not any(y.isalpha() or y.isdigit() for y in x))
punctuation = set(x for x in words_raw if not any(y.isalpha() or y.isdigit() for y in x))

In [13]:
def clean(text):
    return [
        ''.join(x for x in word.lower() if x not in punctuation)
        for word in text
        if word.lower() not in stop_words and word not in punctuation
    ]

train_text = [clean(x) for x in train_text_raw]
train_high = [clean(x) for x in train_high_raw]

In [16]:
vocab = set.union(*[set(x) for x in train_text]) | set.union(*[set(x) for x in train_high])

In [17]:
pickle.dump(train_text, open('train_text.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)
pickle.dump(train_high, open('train_high.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)
pickle.dump(vocab, open('vocab.pickle', 'wb'), protocol = pickle.HIGHEST_PROTOCOL)

## Later things

In [18]:
train_text = pickle.load(open('train_text.pickle', 'rb'))
train_high = pickle.load(open('train_high.pickle', 'rb'))
vocab = list(pickle.load(open('vocab.pickle', 'rb')))

In [53]:
random.sample(vocab, 30)

['malm',
 'dahir',
 'treeform',
 'spectrograph',
 'voas',
 'monstercom',
 'nanotattoo',
 'bowlwinning',
 'yamamota',
 'politicalspeak',
 '2000ft',
 'blann',
 'baganga',
 'westbrook',
 'pradab',
 'semiadolescence',
 'intruder',
 'nazli',
 '56consecutivegame',
 'eishe',
 'extraprecious',
 'schoenkopf',
 '19902009',
 '637pm',
 'peelschools',
 'helstrip',
 'laresident',
 'missives',
 'nullis',
 'wwwhullkrcouk']